In [ ]:
%matplotlib inline
from datetime import datetime
import os

from model import get_1000fps_model, get_basic_unet_model
from data import dataGenerator, load_dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split

import keras
from keras import Model
from keras import backend as K
from keras.layers import Input, Conv2D, MaxPool2D, ZeroPadding2D, Conv2DTranspose, MaxPooling2D
from keras.layers import BatchNormalization, Flatten, Dense, Reshape, concatenate

from keras.models import model_from_json
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow as tf

In [2]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

def extract_contour_mask(mask):
    kernel = np.ones((3,3), np.uint8)
    erosion = cv2.erode(mask, kernel, iterations =1)
    return mask - erosion

def save_test_images(pred,images):
    pred = (pred < 0.5).astype(np.uint8)
    
    results = []
    for i in range(pred.shape[0]):
        mask = extract_contour_mask(pred[i])
        mask = np.stack([mask*255,mask*0,mask*0],axis=-1)
        mask = cv2.resize(mask,(256,256))
        results.append(cv2.add(images[i],mask))
    results = np.concatenate(results)
    
    results = cv2.cvtColor(results,cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(model_dir,"example_output.png"),results)
    
def load_example_images(example_dir):
    images = []
    inputs = []
    for filename in os.listdir(example_dir):
        ex_path = os.path.join(example_dir,filename)
        image = cv2.imread(ex_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        input_ = cv2.resize(image,input_size)
        input_ = input_/255.
        image = cv2.resize(image,(256,256))
        inputs.append(input_)
        images.append(image)

    inputs = np.stack(inputs,axis=0)
    return inputs, images

In [4]:
# set-up

## model setting
filters = 32
depth = 3
input_size = (192,192)
batch_size = 64

model = get_basic_unet_model(input_size, depth=depth, filters=filters)
model.compile("adam",
              loss="mse",
             metrics=['mse',mean_iou])

## test example sample
example_dir = "./data/example/"
inputs, images = load_example_images(example_dir)

## load data generator
dataset = load_dataset(input_size=input_size)
trainset, testset = train_test_split(dataset,test_size=0.1)

train_nums = len(trainset)
train_steps = train_nums // batch_size

test_nums = len(testset)
test_steps = test_nums // batch_size

traingen = dataGenerator(trainset, input_size, batch_size)
testgen = dataGenerator(testset, input_size, batch_size)

tqdm = TQDMNotebookCallback()
model_dir = os.path.join("../save/",datetime.now().strftime("unet_{}_{}-%m%d-%H%M").format(depth,filters))
os.makedirs(model_dir,exist_ok=True)
modelcheckpoint = ModelCheckpoint(os.path.join(model_dir,"model_{val_loss:.4f}.h5"),
                                  monitor='val_loss',
                                 save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss',patience=10)

callbacks= [tqdm, modelcheckpoint, earlystopping]

In [ ]:
hist = model.fit_generator(traingen,
                           train_steps,
                           epochs=100,
                           verbose=0, 
                           validation_data = testgen,
                           validation_steps = test_steps,
                           callbacks=callbacks)

pred = model.predict_on_batch(inputs)
save_test_images(pred,images)

# Draw the results
train_mse = hist.history['mean_squared_error']
valid_mse = hist.history['val_mean_squared_error']
train_iou = hist.history['mean_iou']
valid_iou = hist.history['val_mean_iou']
df = pd.DataFrame({"mse-train":train_mse,
                   "mse-valid":valid_mse,
                   "iou-train":train_iou,
                   "iou-valid":valid_iou})
df.plot().figure.savefig(os.path.join(model_dir,"training.png"))
    
# Save Model
## serialize model to JSON
model_json = model.to_json()
with open(os.path.join(model_dir,"model.json"), "w") as json_file:
    json_file.write(model_json)
## serialize weights to HDF5
model.save_weights(os.path.join(model_dir,"model.h5"))

## Open the file
with open(os.path.join(model_dir,'report.txt'),'w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))

print("Saved model to disk")